# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [50]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [51]:
# Create a reference the CSV file desired
csv_path = "weather_data.csv"

# Read the CSV into a Pandas DataFrame
weather_df = pd.read_csv(csv_path)

# Print the first five rows of data to the screen
weather_df= weather_df.rename(columns={"Wind Speed":"Wind_Speed"})
weather_df.head()

,city,lat,lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,date
0,Hamilton,39.1834,-84.5333,16.11,41,90,5.14,US,1619207506
1,Busselton,-33.6500,115.3333,17.22,72,99,1.34,AU,1619207780
2,Ahipara,-35.1667,173.1667,16.74,79,45,4.45,NZ,1619207620
3,Ormara,25.2088,64.6357,27.46,76,0,5.14,PK,1619207780
4,Beaufort,5.3473,115.7455,27.00,89,20,1.25,MY,1619207781


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [52]:
# Store latitude and longitude in locations
locations = weather_df[["lat", "lng"]]

# Fill NaN values and convert to float
humidity = weather_df["Humidity"].astype(float)


In [53]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity , 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [54]:

weather_check= weather_df.loc[(weather_df['Wind_Speed'] < 10) & (weather_df['Cloudiness'] ==0) & (weather_df['Max_Temp'] > 21.1) &(weather_df['Max_Temp'] < 26.7) ]
weather_check.count()
weather_final=weather_check.dropna()

weather_final.head()

,city,lat,lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,date
88,Pisco,-13.7000,-76.2167,22.00,68,0,3.09,PE,1619207798
95,Cidreira,-30.1811,-50.2056,21.85,76,0,7.75,BR,1619207517
172,Arraial do Cabo,-22.9661,-42.0278,25.00,78,0,8.23,BR,1619207817
246,Morondava,-20.2833,44.2833,25.31,78,0,4.88,MG,1619207596
260,Karratha,-20.7377,116.8463,24.47,31,0,3.60,AU,1619207837


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [55]:
weather_final["Hotel Name"]=""
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in weather_final.iterrows():
    # get lat, lng from df
    lat = row["lat"]
    lng = row["lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
#     print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        weather_final.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
weather_final

,city,lat,lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,date,Hotel Name
88,Pisco,-13.7000,-76.2167,22.00,68,0,3.09,PE,1619207798,Hotel Las Dunas
95,Cidreira,-30.1811,-50.2056,21.85,76,0,7.75,BR,1619207517,Hotel Araçá
172,Arraial do Cabo,-22.9661,-42.0278,25.00,78,0,8.23,BR,1619207817,Ocean View Hotel
246,Morondava,-20.2833,44.2833,25.31,78,0,4.88,MG,1619207596,Palissandre Cote Ouest resort & SPA
260,Karratha,-20.7377,116.8463,24.47,31,0,3.60,AU,1619207837,Karratha International Hotel
318,Rocha,-34.4833,-54.3333,21.58,73,0,4.64,UY,1619207846,La Costa Hotel
449,Akālgarh,29.8167,75.8833,21.57,49,0,2.50,IN,1619207883,Hotel KT Royal Sangrur
465,Wer,27.0000,77.1667,25.72,37,0,4.60,IN,1619207887,The Bagh
503,Cristalina,-16.7686,-47.6136,23.27,48,0,1.89,BR,1619207896,Ity Hotel
516,San Juan,-31.5375,-68.5364,23.61,33,0,6.37,AR,1619207810,Hotel Viñas del Sol


In [56]:
hotel_df = weather_final[["Hotel Name","city","Country","lat","lng"]]

hotel_df

,Hotel Name,city,Country,lat,lng
88,Hotel Las Dunas,Pisco,PE,-13.7000,-76.2167
95,Hotel Araçá,Cidreira,BR,-30.1811,-50.2056
172,Ocean View Hotel,Arraial do Cabo,BR,-22.9661,-42.0278
246,Palissandre Cote Ouest resort & SPA,Morondava,MG,-20.2833,44.2833
260,Karratha International Hotel,Karratha,AU,-20.7377,116.8463
318,La Costa Hotel,Rocha,UY,-34.4833,-54.3333
449,Hotel KT Royal Sangrur,Akālgarh,IN,29.8167,75.8833
465,The Bagh,Wer,IN,27.0000,77.1667
503,Ity Hotel,Cristalina,BR,-16.7686,-47.6136
516,Hotel Viñas del Sol,San Juan,AR,-31.5375,-68.5364


In [57]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lng"]]

In [61]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
markers = gmaps.marker_layer(locations,info_box_content = hotel_info)
fig.add_layer(markers)


# Display figure
fig



Figure(layout=FigureLayout(height='420px'))

In [62]:
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))